In [1]:
# 1. 라이브러리 설치
!pip install transformers datasets accelerate pandas scikit-learn

import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# ==========================================
# 1. 설정 및 데이터 로드 함수
# ==========================================
MODEL_NAME = "beomi/kcbert-large"
MAX_LEN = 128
BATCH_SIZE = 8  # Large 모델 메모리 고려

def load_kocosa_json(file_path):
    try:
        # 1. JSON 읽기
        df = pd.read_json(file_path)
    except ValueError:
        # JSONL 등의 포맷일 경우 대비
        df = pd.read_json(file_path, lines=True)

    # 2. 데이터 전처리 (BERT 입력 포맷 만들기)
    # Context + [SEP] + Response
    df['full_text'] = df['Context'] + " [SEP] " + df['Response']

    # 3. 라벨 인코딩 (Sarcasm -> 1, Non-Sarcasm -> 0)
    # 추론용 데이터에 라벨이 없어도 에러 안 나게 처리
    if 'Sarcasm_Label' in df.columns:
        df['label'] = df['Sarcasm_Label'].apply(lambda x: 1 if "Sarcasm" in str(x) and "Non" not in str(x) else 0)
    else:
        df['label'] = 0 # 라벨 없으면 더미값 0

    return df

# ==========================================
# 2. 데이터 로드 (파일 경로 수정됨)
# ==========================================
print("📉 학습 데이터(KoCoSa Train) 로딩 중...")
train_df = load_kocosa_json('KoCoSa_train.json')
eval_df = load_kocosa_json('KoCoSa_test.json').head(100) # 검증용으로 조금만 사용


inference_file = 'movie_reviews_demo.json'
try:
    inference_df = load_kocosa_json(inference_file)
    print(f"✅ 추론용 데이터 {len(inference_df)}개 로드 완료!")
except ValueError as e:
    print(f"❌ 파일을 찾을 수 없거나 형식이 잘못되었습니다: {inference_file}")
    # 테스트용 빈 데이터프레임
    inference_df = pd.DataFrame({'Context':['Test'], 'Response':['Test'], 'Sarcasm_Label':['Non-Sarcasm']})
    inference_df['full_text'] = "Test [SEP] Test"
    inference_df['label'] = 0

# ==========================================
# 3. 데이터셋 클래스 정의
# ==========================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class SarcasmDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['full_text'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        encoding = self.tokenizer(
            str(self.texts[item]),
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[item], dtype=torch.long)
        }

# ==========================================
# 4. 모델 학습 (Fine-tuning)
# ==========================================
print(f"\n🚀 {MODEL_NAME} 모델 파인튜닝 시작...")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = SarcasmDataset(train_df, tokenizer, MAX_LEN)
eval_dataset = SarcasmDataset(eval_df, tokenizer, MAX_LEN)

training_args = TrainingArguments(
    output_dir='./results_kcbert_json',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    logging_steps=50,
    save_strategy="no",
    eval_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()
print("✅ 모델 학습 완료!")

# ==========================================
# 5. 추론 및 결과 출력
# ==========================================
print("\n🔍 JSON 데이터 분석(Inference) 시작...")

# 추론 데이터셋 준비
inference_dataset = SarcasmDataset(inference_df, tokenizer, MAX_LEN)

# 예측 실행
predictions = trainer.predict(inference_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=-1)

# 결과 정리
results = []
for i, row in inference_df.iterrows():
    is_sarcasm = preds[i] == 1
    sarcasm_prob = probs[i][1].item()

    # JSON 파일에 있는 원래 라벨(점수 기반)도 같이 표시해서 비교
    original_label = row.get('Sarcasm_Label', 'Unknown')

    results.append({
        'Review': row['Response'],
        'Original_Label': original_label, # 점수 기반 라벨 (참고용)
        'Model_Pred': "Sarcasm(풍자) ⭕" if is_sarcasm else "Normal(일반) ❌",
        'Confidence': f"{sarcasm_prob*100:.1f}%"
    })

result_df = pd.DataFrame(results)

# 결과 CSV 저장
result_df.to_csv("kcbert_large_json_inference_result.csv", index=False, encoding='utf-8-sig')

print("\n" + "="*60)
print("📊 분석 결과 (상위 10개)")
print("="*60)
# 전체 컬럼 출력 설정
pd.set_option('display.max_colwidth', 50)
print(result_df[['Review', 'Original_Label', 'Model_Pred', 'Confidence']].head(10))

# 'Sarcasm'으로 예측된 건수 확인
sarcasm_count = result_df['Model_Pred'].str.contains("풍자").sum()
print(f"\n🔥 총 {len(result_df)}개 중 {sarcasm_count}개를 '풍자'로 예측했습니다.")

📉 학습 데이터(KoCoSa Train) 로딩 중...
📈 추론 데이터(Real Movie JSON) 로딩 중...
✅ 추론용 데이터 10개 로드 완료!


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🚀 beomi/kcbert-large 모델 파인튜닝 시작...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,0.705400
100,0.706400
150,0.670100
200,0.634300
250,0.615500
300,0.589600
350,0.588100
400,0.592600
450,0.569600
500,0.592100


✅ 모델 학습 완료!

🔍 JSON 데이터 분석(Inference) 시작...


In [9]:
# 1. 모델이 좋아하는 '줄바꿈(\n)'이 들어간 대화 형식을 하드코딩합니다.
# (함수 안 쓰고 그냥 문자열 변수에 넣습니다)
formatted_context = "B: 야, 이번에 개봉한 영화 '리얼' 봤어?\nA: 응 봤지. 근데 그거 평점 테러 당하고 완전 망작이라던데.\nB: 진짜? 그 정도야? 보고 나서 어땠는데?"

# 2. 데이터프레임의 Context를 이 형식으로 싹 덮어씌웁니다.
inference_df['Context'] = formatted_context

# 3. 모델 입력 형식(Full Text) 만들기
inference_df['full_text'] = inference_df['Context'] + " [SEP] " + inference_df['Response']

# 4. 추론 실행
inference_dataset = SarcasmDataset(inference_df, tokenizer, MAX_LEN)
predictions = trainer.predict(inference_dataset)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=-1)

# 5. 결과 확인 (이제 점수가 팍 튈 겁니다)
print("\n=== 🎯 인코딩(줄바꿈) 적용 후 결과 ===")
for i in range(10):
    review = inference_df.iloc[i]['Response']
    score = probs[i][1].item() * 100
    print(f"리뷰: {review[:15]}... | 반어법 확률: {score:.1f}%")
    results = []
for i, row in inference_df.iterrows():
    is_sarcasm = preds[i] == 1
    sarcasm_prob = probs[i][1].item()

    # JSON 파일에 있는 원래 라벨(점수 기반)도 같이 표시해서 비교
    original_label = row.get('Sarcasm_Label', 'Unknown')

    results.append({
        'Review': row['Response'],
        'Original_Label': original_label, # 점수 기반 라벨 (참고용)
        'Model_Pred': "Sarcasm(풍자) ⭕" if is_sarcasm else "Normal(일반) ❌",
        'Confidence': f"{sarcasm_prob*100:.1f}%"
    })

result_df = pd.DataFrame(results)


=== 🎯 인코딩(줄바꿈) 적용 후 결과 ===
리뷰: 살다살다 이런 전위적인 괴작... | 반어법 확률: 28.2%
리뷰: 옆에서 폰을 해도 화나지 않... | 반어법 확률: 17.2%
리뷰: 진짜 2017년 최악의 영화... | 반어법 확률: 33.7%
리뷰: 이것을 보느니 집에서 천장만... | 반어법 확률: 70.3%
리뷰: 나만 당할순 없다 ....... | 반어법 확률: 0.3%
리뷰: 제가 팝콘먹으면서봐도 사람들... | 반어법 확률: 0.2%
리뷰: 에휴... 감독이 이 배우들... | 반어법 확률: 11.7%
리뷰: 아..그냥 아....... | 반어법 확률: 0.5%
리뷰: 내용도없고. .팩트도없고..... | 반어법 확률: 0.7%
리뷰: 김수현 성동일 연기는 진짜 ... | 반어법 확률: 0.2%
